## Request and Inspect Weekly SPX-Options

First, load ib-ruby environment

In [1]:
require_relative 'lib/iruby_init.rb'


----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Check the Presence of predefined indices

In [2]:
Symbols::Index.all

[:a_d, :asx, :dax, :hsi, :minihsi, :spx, :stoxx, :tick, :trin, :vasx, :vdax, :vhsi, :vix, :volume, :vstoxx]

In [3]:
spx_market_price = IB::Symbols::Index.spx.market_price

TWS Warning 2119: Market data farm is connecting:usfarm.nj
TWS Warning 2104: Market data farm connection is OK:usfarm.nj
TWS Error 354: Requested market data is not subscribed.Delayed market data is available.Error&CBOE/IND/Top&CBOE/IND/Top
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.


4224.51

_Note:_ If the request is fired before the market opening, the returned market price is the closing price of the previous day.

Next we define a range of strikes to consider as  4215 to 4340

In [15]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, expiry: 20210609, 
                   exchange: :smart, right: :put


spxw_options = spxw.verify{ |c| c if c.strike > spx_market_price - 15 && c.strike < spx_market_price + 15  }.compact
spxw_options.map(&:strike).sort

[4210.0, 4215.0, 4220.0, 4225.0, 4230.0, 4235.0]

Lets inspect these Options further.

The method _request_greeks_ requests Tickdata from the TWS-Server and collects the calculated greeks 


In [14]:
spxw_options.map( &:request_greeks )
puts "-"*50
puts spxw_options.sort_by(&:strike).map{|x| [x.to_human,x.greek.to_human].join( '-->')}.join("\n")

TWS Warning 2119: Market data farm is connecting:usopt
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 2119: Market data farm is connecting:usopt
TWS Warning 10197: No market data during competing live session
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 2119: Market data farm is connecting:usopt
TWS Warning 10197: No market data during competing live session
TWS Warning 2103: Market data farm connection is broken:usopt
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 2119: Market data farm is connecting:usopt
TWS Warning 2119: Market data farm is connectin

### Get ATM-Weeklies

There is a method `Contract#atm-options`. Its designed for monthly options

In [17]:
atm_options =  Symbols::Index.spx.atm_options

atm_options.keys    # Expiries of the Options

[2106, 2107, 2108, 2109, 2111, 2112, 2201, 2202, 2203, 2205, 2206, 2212, 2301]

In [18]:
atm_options[2106].map &:to_human

["<Option: SPX 20210617 put 4235.0 SMART USD>"]

To get all available ATM-SPXW-Options we define a sample (as above) and specify the strike

In [27]:
spxw =  Option.new trading_class: :spxw, symbol: :spx, currency: :usd, strike: 4230, 
                   exchange: :smart, right: :put

spxw_chain =  spxw.verify.sort_by &:last_trading_day
puts spxw_chain.map &:to_human


<Option: SPX 20210607 put 4230.0 SMART USD>
<Option: SPX 20210609 put 4230.0 SMART USD>
<Option: SPX 20210611 put 4230.0 SMART USD>
<Option: SPX 20210614 put 4230.0 SMART USD>
<Option: SPX 20210616 put 4230.0 SMART USD>
<Option: SPX 20210618 put 4230.0 SMART USD>
<Option: SPX 20210621 put 4230.0 SMART USD>
<Option: SPX 20210623 put 4230.0 SMART USD>
<Option: SPX 20210625 put 4230.0 SMART USD>
<Option: SPX 20210628 put 4230.0 SMART USD>
<Option: SPX 20210630 put 4230.0 SMART USD>
<Option: SPX 20210702 put 4230.0 SMART USD>
<Option: SPX 20210706 put 4230.0 SMART USD>
<Option: SPX 20210707 put 4230.0 SMART USD>
<Option: SPX 20210709 put 4230.0 SMART USD>
<Option: SPX 20210716 put 4230.0 SMART USD>
<Option: SPX 20210723 put 4230.0 SMART USD>
<Option: SPX 20210730 put 4230.0 SMART USD>
<Option: SPX 20210820 put 4230.0 SMART USD>
<Option: SPX 20210831 put 4230.0 SMART USD>
<Option: SPX 20210917 put 4230.0 SMART USD>
<Option: SPX 20210930 put 4230.0 SMART USD>
<Option: SPX 20211015 put 4230.0

In [30]:
spxw_chain.each{|o| o.request_greeks( thread: true)}.join
puts spxw_chain.map{|o| "Expiry: #{o.expiry}   Market-price: #{o.greek.to_human}"}.join("\n")

Expiry: 20210607   Market-price: < Greeks::  delta:  --; gamma: --, vega: --; theta: -- >
Expiry: 20210609   Market-price: <  optionPrice: 8.83065, UnderlyingPrice: 4227.88 impl.Vola: 0.0848778 ; dividend: 0.498997; Greeks::  delta:  -0.554505; gamma: 0.0211502, vega: 0.870071; theta: -- >
Expiry: 20210611   Market-price: <  optionPrice: 18.2931, UnderlyingPrice: 4227.88 impl.Vola: 0.107186 ; dividend: 1.64098; Greeks::  delta:  -0.534248; gamma: 0.0096946, vega: 1.52035; theta: -- >
Expiry: 20210614   Market-price: <  optionPrice: 22.0263, UnderlyingPrice: 4227.88 impl.Vola: 0.0927843 ; dividend: 1.78797; Greeks::  delta:  -0.528079; gamma: 0.00792237, vega: 2.15445; theta: -1.6628 >
Expiry: 20210616   Market-price: <  optionPrice: 29.4508, UnderlyingPrice: 4227.88 impl.Vola: 0.109626 ; dividend: 2.06195; Greeks::  delta:  -0.520555; gamma: 0.00581279, vega: 2.49097; theta: -1.70311 >
Expiry: 20210618   Market-price: <  optionPrice: 35.1358, UnderlyingPrice: 4227.88 impl.Vola: 0.11809

TWS Error 354: Requested market data is not subscribed.Delayed market data is not available.SPX (SPXW) JUN 21 '21 4230 Put (SPXW  210621P04230000) /TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10167: Requested market data is not subscribed. Displaying delayed market data.
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.Delayed market data is available.SPX S&P 500 Stock Index/TOP/ALL
TWS Warning 10090: Part of requested market data is not subscribed. Subscription-independent ticks are still active.D

TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 10197: No market data during competing live session
TWS Warning 2108: Market data farm connection is inactive but should be available upon demand.usfarm.nj
TWS Warning 2108: Market data farm connection is inactive but should be available upon demand.usfarm.nj
